In [30]:
from langchain_core.tools import tool
from typing import List
from langchain_openai.chat_models import ChatOpenAI
from pydantic import BaseModel
import dotenv
import os

dotenv.load_dotenv()

True

In [16]:
thought_raw_text = "One of the dogs I had growing up was named Ellie. She was a basenji and she didnt like people very much"

In [144]:
model = ChatOpenAI(model="gpt-3.5-turbo-1106")
model.temperature = 0.0


In [160]:
from typing import Optional

class Character(BaseModel):
    """
    A non-narrator character in a story.

    Attributes:
    - name: The name of the character.
    - relationship: A description of the character in relation to the narrator of a story, written in a third person perspective without explicitly mentioning the narratator. For example: "Best Friend", or "deceased grandmother"
    - description: A short, accurate description of the character from the perspective of the narrator of the story, written in a third person perspective.

    """
    name: str
    relationship: str
    description: str

# @tool
# def thought_to_characters(thought: str) -> List[]:
#     """
#     extract character.
#     """
#     return [thought]

In [161]:
model_with_tools = model.bind_tools([Character])

In [88]:
model_with_tools.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'Character',
   'description': 'A character in a story. Attributes:\n- name: The name of the character.\n- relationship: A description of the character in relation to the narrator of a story.\n- description: A description of the chracter from the perspective of the narrator of the story.',
   'parameters': {'type': 'object',
    'properties': {'name': {'type': 'string'},
     'relationship': {'type': 'string'},
     'description': {'type': 'string'}},
    'required': ['name', 'relationship', 'description']}}}]

In [163]:
from langchain.output_parsers import JsonOutputToolsParser
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    template="""
    You are provided with a story or thought from the perspective of a narrator.
    Describe the non-narrator characters in the story or thought.

    Use only information provided in the story or thought to describe the characters. Do not make any assumptions about the characters' personalities or add any additional information about the characters not explicitly mentioned in the thought.
    Write any descriptions from a third-person perspective.
    
    Story:
    {thought}
    """
)
thought = "I lived in a house during the COVID-19 pandemic with 4 housemates. Emily, Isobelle, Tessa, and Jo were all grad students at Cal Poly SLO and they made a great community in an otherwise isolated time."


chain = prompt | model_with_tools | JsonOutputToolsParser()

chain.invoke({"thought":thought})

[{'args': {'name': 'Emily',
   'relationship': 'Housemate',
   'description': 'Emily was a dedicated and hardworking grad student, always focused on her studies and research.'},
  'type': 'Character'},
 {'args': {'name': 'Isobelle',
   'relationship': 'Housemate',
   'description': 'Isobelle was a cheerful and optimistic grad student, bringing positivity and laughter to the house.'},
  'type': 'Character'},
 {'args': {'name': 'Tessa',
   'relationship': 'Housemate',
   'description': 'Tessa was a kind and empathetic grad student, always ready to lend a listening ear and offer support to her housemates.'},
  'type': 'Character'},
 {'args': {'name': 'Jo',
   'relationship': 'Housemate',
   'description': 'Jo was a creative and adventurous grad student, often coming up with fun and innovative ideas to keep the housemates entertained.'},
  'type': 'Character'}]

In [61]:
model_with_tools.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'Character',
   'description': 'A character in a story.',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string'}},
    'required': ['name', 'description']}}}]

In [141]:
import asyncio
import time

async def my_coroutine(n):
    await asyncio.sleep(n)
    print(f'Coroutine {n} finished')

async def main():
    # Schedule my_coroutine to run and immediately continue with the next line
    task = asyncio.create_task(my_coroutine(2))

    print('Doing something else in the main function')

    # Wait for my_coroutine to finish
    await task
    
await main()

Doing something else in the main function
Coroutine 2 finished
